In [ ]:
# import necessary packages and utils

import os
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F

from torchvision import models, transforms, datasets

import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset

import PIL.Image as Image
from tqdm import tqdm
import pickle

from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error as mse
from sklearn import linear_model
from sklearn.model_selection import StratifiedShuffleSplit

from models.vgg16 import *

In [ ]:
# config   
num_epochs = 1
num_classes = 2
batch_size = 16
learning_rate = 1e-5
# model_name = vgg16_bn(pretrained=True) # baseline model

In [ ]:
# load cvs data

data_dir = "../data/petfinder-pawpularity-score/"

img_train_dir = os.path.join(data_dir, 'train')

def return_imgfilepath(name, folder=img_train_dir):
    path = os.path.join(folder, f'{name}.jpg')
    return path

train_file_path = os.path.join(data_dir, 'train.csv')
train_df = pd.read_csv(train_file_path)

# set image filepath
train_df['file_path'] = train_df['Id'].apply(lambda x: return_imgfilepath(x))
train_df.head()

In [ ]:
im = Image.open(train_df['file_path'][0])
# im.show()

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
device

In [ ]:
class PetDataset(Dataset):
    def __init__(self, image_filepaths, targets, transform=None):
        self.image_filepaths = image_filepaths
        self.targets = targets
        self.transform = transform
    
    def __len__(self):
        return len(self.image_filepaths)

    def __getitem__(self, idx):
        image_filepath = self.image_filepaths[idx]
        with open(image_filepath, 'rb') as f:
            image = Image.open(f)
            image_rgb = image.convert('RGB')
        # image = np.array(image_rgb) / 255 # convert to 0-1

        if self.transform is not None:
            image = self.transform(image)
        
#         image = torch.transpose(image, (2, 0, 1))
        target = self.targets[idx]

        image = torch.tensor(image)
        target = torch.tensor(target)
        
        return image, target

In [ ]:
images = np.array(train_df['file_path'])
targets = np.array(train_df['Pawpularity'])

In [ ]:
images.shape, targets.shape

In [ ]:
# get data
# train_loader, val_loader, test_loader = load_data(batch_size, use_subset=True)

def load_data(batch_size, use_subset=True):
    """
    return the train/val/test dataloader
    """
    
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5] * 3, std=[0.5] * 3)
    ])
    
    train_dataset = PetDataset(image_filepaths=images, targets=targets, transform=transform)
#     val_dataset = PetDataset(image_filepaths=images, targets=targets, transform=transform)
#     test_dataset = PetDataset(image_filepaths=images, targets=targets, transform=transform)
    
    indices_train = list(range(700))
#     indices_val = list(range(150))    
#     indices_test = list(range(150))
    
    train_subset = Subset(train_dataset, indices_train)
#     val_subset = Subset(train_dataset, indices_val)
#     test_subset = Subset(test_dataset, indices_test)

    # data loader
    train_loader = DataLoader(dataset=train_subset if use_subset else train_dataset, 
                                batch_size=batch_size,
                                shuffle=True)
#     val_loader = DataLoader(dataset=val_subset if use_subset else val_dataset,
#                                 batch_size=batch_size,
#                                 shuffle=False)
#     test_loader = DataLoader(dataset=test_subset if use_subset else test_dataset,
#                                 batch_size=batch_size,
#                                 shuffle=False)
    
#     return train_loader, val_loader, test_loader
    return train_loader

train_loader = load_data(batch_size, use_subset=True)

In [ ]:
# init model

class ConvNet_v1(nn.Module):
    """
    Simple two-layer CNN with sequential container
    """
    def __init__(self):
        super(ConvNet_v1, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, 3),
            nn.ReLU(),
            nn.Conv2d(16, 32, 3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Flatten()
        )
        self.layer2 = nn.Sequential(
            nn.Linear(387200, 128),
            nn.ReLU(),
            nn.Linear(128, 1) 
        )

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out).squeeze()
        return out

In [ ]:
# init model

def initialize_model(model, learning_rate, num_classes):
    """
    initialize the model (pretrained vgg16_bn)
    define loss function and optimizer and move data to gpu if available
    
    return:
        model, loss function(criterion), optimizer
    """
    
#     num_ftrs = model.classifier[6].in_features
#     model.classifier[6] = nn.Linear(num_ftrs, num_classes)
    model = model.to(device)
    
    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()   # potential alternative: nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    return model, criterion, optimizer

model_name = ConvNet_v1()

model, criterion, optimizer = initialize_model(model_name, learning_rate, num_classes)

In [ ]:
experiment_name = "simple_run"

In [ ]:
# train

def calc_rmse(y_pred, y_true):
    return np.sqrt(((y_pred - y_true) ** 2).mean())


def train(train_loader, model, criterion, optimizer, num_epochs):
    """
    Move data to GPU memory and train for specified number of epochs
    Also plot the loss function and save it in `Figures/`
    Trained model is saved as `cnn.ckpt`
    """
    for epoch in range(num_epochs): # repeat the entire training `num_epochs` times
        # for each training sample
        loss_hist = []
        step_hist = []
        for i, (images, label) in (enumerate(train_loader)):
            
            model.train()
            train_pred = list()
            train_true = list()
            
            # move to gpu if available
            images = images.to(device).float()
            label = label.to(device).float()
                
            # forward pass
            out = model(images)
            loss_func = nn.MSELoss()
            loss = torch.sqrt(loss_func(out, label))
            
            print(type(loss))

            # backprop
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_true += label.cpu().detach().numpy().tolist()
            train_pred += out.cpu().detach().numpy().tolist()

            train_rmse = calc_rmse(np.array(train_pred), np.array(train_true))
        
            print('Train rmse: {}'.format(train_rmse))
        
    torch.save(model.state_dict(), experiment_name+'.ckpt')

In [ ]:
train(train_loader, model, criterion, optimizer, num_epochs)

In [ ]:
# eval
